In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

### Téléchargement du df

In [ ]:
#Dataset Champs Elysées
raw_df = pd.read_csv(r'data\comptages-routiers-permanents-washington-filtered.csv', delimiter=';')

# Convertir la colonne en datetime et définir l'index
raw_df['Date et heure de comptage'] = pd.to_datetime(raw_df['Date et heure de comptage'], errors='coerce', utc=True)
raw_df.set_index('Date et heure de comptage', inplace=True)
raw_df.sort_index(inplace=True)

### Distribution des valeurs manquantes 'Débit horaire' est NaN

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Filtrer les dates où 'Débit horaire' est NaN
nan_dates = raw_df.index[raw_df['Débit horaire'].isna()]  # Index contient les dates

# Convertir en série Pandas pour le traitement
nan_dates_series = pd.Series(nan_dates)

# Tracer un histogramme des fréquences des dates
plt.figure(figsize=(10, 6))
nan_dates_series.hist(bins=30, color='skyblue', edgecolor='black')
plt.title('Histogramme des dates où Débit horaire est NaN')
plt.xlabel('Dates')
plt.ylabel('Nombre de valeurs NaN')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Visualisation de l'évolution du débit horaire et taux d'occupation sur un jour donné

In [ ]:
# Sélectionner la portion du DataFrame pour la période donnée
sept_debit_df = raw_df.loc['2023-12-17 00:00:00+00:00':'2023-12-17 23:00:00+00:00']


# Créer deux graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharex=True)

# Graphique 1 : Débit horaire
axs[0].plot(sept_debit_df.index, sept_debit_df['Débit horaire'], label='Débit horaire', color='blue')
axs[0].set_title('Évolution du débit horaire')
axs[0].set_xlabel('Heure')
axs[0].set_ylabel('Débit')
axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  # Format heure:minute
axs[0].xaxis.set_major_locator(mdates.HourLocator(interval=1))  # Tick toutes les heures
axs[0].tick_params(axis='x', rotation=45)
axs[0].grid(True)
axs[0].legend()

# Graphique 2 : Taux d'occupation
axs[1].plot(sept_debit_df.index, sept_debit_df['Taux d\'occupation'], label='Taux d\'occupation', color='orange')
axs[1].set_title('Évolution du taux d\'occupation')
axs[1].set_xlabel('Heure')
axs[1].set_ylabel('Taux d\'occupation')
axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  # Format heure:minute
axs[1].xaxis.set_major_locator(mdates.HourLocator(interval=1))  # Tick toutes les heures
axs[1].tick_params(axis='x', rotation=45)
axs[1].grid(True)
axs[1].legend()

# Ajuster la mise en page
plt.tight_layout()
plt.show()


### Attribution à chaque heure une période de la journée

In [ ]:
# S'assurer que l'index est de type datetime
raw_df.index = pd.to_datetime(raw_df.index)

# Définir une fonction pour classer les périodes
def assigner_periode(hour):
    if 0 <= hour < 6:
        return "Nuit"
    elif 6 <= hour < 9:
        return "Matin calme"
    elif 9 <= hour < 12:
        return "Matinée"
    elif 12 <= hour < 16:
        return "Après-midi"
    elif 16 <= hour < 22:
        return "Soirée"
    else:
        return "Nuit tardive"

# Appliquer la fonction pour ajouter une colonne "Période"
raw_df['Période'] = raw_df.index.hour.map(assigner_periode)

# Afficher les premières lignes pour vérifier

raw_df.groupby('Période')["Taux d'occupation"].mean().plot(kind='bar', xlabel='Jour', ylabel="Taux d'occupation", color='skyblue', edgecolor='black', figsize=(10, 6))

In [ ]:
weather_df = pd.read_csv(r'data\cleaned_weather.csv', delimiter=',', index_col=2)

In [ ]:
# Convertir l'index en type datetime si ce n'est pas déjà fait
weather_df.index = pd.to_datetime(weather_df.index)
raw_df.index = pd.to_datetime(raw_df.index)

# Fonctions pour récupérer les données à partir de weather_df
def concat_prcp(date):
    return weather_df.loc[date, 'PRCP'] if date in weather_df.index else None

def concat_snwd(date):
    return weather_df.loc[date, 'SNWD'] if date in weather_df.index else None

def concat_tmax(date):
    return weather_df.loc[date, 'TMAX'] if date in weather_df.index else None

# Appliquer les fonctions pour ajouter des colonnes à raw_df
raw_df['PRCP'] = raw_df.index.map(lambda x: concat_prcp(x.strftime('%Y-%m-%d')))
raw_df['SNWD'] = raw_df.index.map(lambda x: concat_snwd(x.strftime('%Y-%m-%d')))
raw_df['TMAX'] = raw_df.index.map(lambda x: concat_tmax(x.strftime('%Y-%m-%d')))


### Nouveau df (4 nouvelles features : période, PRCP, SNWD et TMAX)

In [ ]:
# Afficher raw_df
raw_df.head(25)